# Task 2: Rescue Across The Rift

Our task is to first perform data cleaning on our given dataset that was partially corrupted due to a space anomaly that transported passengers from the legendary Star Trek passenger liner into unknown regions of space. We need to clean the dataset and provide a clean dataset as output.

For the second part of the task, we need to perform exploratory data analysis on our cleaned dataset to try to find correlations about why these passengers might have been transported by the anomaly and learn more about the incident and possible status of passengers.

In [1]:
#import necessary libaries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

In [2]:
#Get and check data
data = pd.read_csv('../Data/train.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [3]:
#Try to get an idea about the data
data.head(10)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
5,0005_01,Earth,False,F/0/P,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.0,Sandie Hinetthews,True
6,0006_01,Earth,False,F/2/S,TRAPPIST-1e,26.0,False,42.0,1539.0,3.0,0.0,0.0,Billex Jacostaffey,True
7,0006_02,Earth,True,G/0/S,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,NaN,Candra Jacostaffey,True
8,0007_01,Earth,False,F/3/S,TRAPPIST-1e,35.0,False,0.0,785.0,17.0,216.0,0.0,Andona Beston,True
9,0008_01,Europa,True,B/1/P,55 Cancri e,14.0,False,0.0,0.0,0.0,0.0,0.0,Erraiam Flatic,True


## Step 1: Data Cleaning

We will take a columnwise approach to data cleaning, and slowly cover all columns and try to clean the whole dataset that way in this case

### 1) Cleaning Passenger_Id and Name

We can clearly see some correlation between passenger Id and 'family' since 0003_01 and _02 correlate to people with same last name (and there is another example of the same in the 10 above as well)

This implies that passenger Id is divided into 2 parts:
- First part signifies family number in the ship (same last name)
- Second part signifies the name of the people in the family (possibly in alphabetical order)

In [4]:
#Let's first check for NaN values in passener Id and Name columns

print(f"Number of NaN values in passengerId is {data['PassengerId'].isna().sum()}")
print(f"Number of NaN values in Name is {data['Name'].isna().sum()}")


Number of NaN values in passengerId is 0
Number of NaN values in Name is 200


In [5]:
#Check if more likely to have NaN name if transported?

(data[data['Name'].isna()].loc[:, ['Transported']] == True).sum()
#Not more likely by the results

Transported    101
dtype: int64

In [6]:
#Just because passengerId doesn't have NaN values doesn't mean its clear
# let's check for duplicate values next, as that should not be allowed

data[data['PassengerId'].duplicated() == True]

#Ok good no duplicate values here
#I think this means that the whole PassengerId column should be clean already

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported


In [7]:
#Now let's see what's up with the NaN values of names

data[data['Name'].isna()]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
27,0022_01,Mars,False,D/0/P,TRAPPIST-1e,21.0,False,980.0,2.0,69.0,0.0,0.0,NaN,False
58,0064_01,Mars,True,F/14/S,TRAPPIST-1e,15.0,False,0.0,0.0,0.0,0.0,0.0,NaN,True
65,0069_01,Earth,False,F/16/S,TRAPPIST-1e,42.0,False,887.0,0.0,9.0,6.0,0.0,NaN,True
77,0082_03,Mars,False,F/16/P,TRAPPIST-1e,8.0,False,0.0,0.0,0.0,0.0,0.0,NaN,True
101,0108_02,Earth,False,G/19/S,TRAPPIST-1e,31.0,False,562.0,0.0,326.0,0.0,0.0,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8629,9205_02,Europa,True,B/300/P,TRAPPIST-1e,15.0,False,0.0,0.0,0.0,0.0,0.0,NaN,True
8631,9208_01,Earth,True,G/1485/S,TRAPPIST-1e,35.0,False,0.0,0.0,0.0,0.0,0.0,NaN,True
8636,9218_01,Europa,True,B/353/S,55 Cancri e,43.0,False,0.0,0.0,0.0,0.0,0.0,NaN,True
8652,9230_01,Europa,False,C/342/S,TRAPPIST-1e,36.0,True,0.0,5600.0,715.0,2868.0,971.0,NaN,True


Hmm no explicit pattern seen here, maybe they are just completely random NaN values (which we cant fill in any way). Anyways, the name column in itself might be useless as the name itself doesnt matter

The passengerId has the 'family' and 'name' info in itself and the actual name doesn't matter.
We will probably end up dropping the name column entirely at the end of cleaning since it seems to be irrelevant in every way

What is interesting lies in our splitting passengerId into familyNo and nameNo idea.

First let's check whether our theory that people with same 'familyId' will have same 'last name', since that would confirm that that is how the Id works.


In [8]:
#First let's check how many such values there are (how many people part of 'families)

data['PassengerId'].str[:4].duplicated(keep = False).sum()
#Damn quite a lot

np.int64(3888)

In [9]:
#What we want to check:
#For every value of PassengerId with same first 4 starting letters, the corresponding Name's 2nd part (last name) is same

#We first create a dataset with boolean masking that returns all rows part of a 'family'
#Then we group by the first 4 letters (familyCode) 
#We apply a filter that checks if all people in a family have same last name
#We do this by splitting Name string into 2 and taking the last name
#And then we check for number of 'unique' last names, which should just be 1, if they all have the same last name
#So we filter to check for cases where the number of unique last names of all members in a group is NOT 1

data.groupby(data[data['PassengerId'].str[:4].duplicated(keep = False)]['PassengerId'].str[:4]).filter(lambda x: x['Name'].str.split().str[1].nunique() != 1)

#Welp there goes our theory (966 values that dont follow our theory spotted)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
21,0020_01,Earth,True,E/0/S,TRAPPIST-1e,1.0,False,0.0,0.0,0.0,0.0,0.0,Almary Brantuarez,False
22,0020_02,Earth,True,E/0/S,55 Cancri e,49.0,False,0.0,0.0,0.0,0.0,0.0,Glendy Brantuarez,False
23,0020_03,Earth,True,E/0/S,55 Cancri e,29.0,False,0.0,0.0,NaN,0.0,0.0,Mollen Mcfaddennon,False
24,0020_04,Earth,False,E/0/S,TRAPPIST-1e,10.0,False,0.0,0.0,0.0,0.0,0.0,Breney Jacostanley,True
25,0020_05,Earth,True,E/0/S,PSO J318.5-22,1.0,False,NaN,0.0,0.0,0.0,0.0,Mael Brantuarez,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8577,9157_06,Earth,False,G/1476/S,TRAPPIST-1e,12.0,False,0.0,0.0,0.0,0.0,0.0,Pamont Navages,False
8578,9157_07,Earth,True,G/1476/S,TRAPPIST-1e,3.0,False,0.0,0.0,0.0,0.0,0.0,Racey Navages,True
8639,9220_01,Earth,False,G/1496/P,TRAPPIST-1e,25.0,False,2.0,45.0,45.0,0.0,815.0,Branca Meyerthy,False
8640,9220_02,Earth,True,G/1496/P,TRAPPIST-1e,20.0,False,0.0,0.0,0.0,0.0,0.0,Frey Meyerthy,True


So this proves that groups are not based on direct family, so names and passengerId have no concrete connection as same group number will not imply same last name. Maybe these groups are family + friends/family + distant relatives, whatever the case may be, this observation makes our 'name' column completely useless at it is not connected to passengerId and has so many NaN values.

Thus we will drop the 'Name' column completely near the end of cleaning

In [10]:
# PassengerId contains GroupId which is important information to try to explore if people in same group were more likely to be transported or something
# PersonId (for the people in a group) might not be important
# But groupSize (number of people in group) could be an influential factor to explore

#We aim to add 2 new columns to a new df (final cleaned version), one is groupId and other is groupSize

cleanData = data.copy()

#Creating groupId column
cleanData['GroupId'] = data['PassengerId'].str[:4]
cleanData.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,GroupId
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,0001
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,0002
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,0003
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,0003
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,0004


In [11]:
# Creating groupSize column

cleanData['GroupSize'] = data.groupby(data['PassengerId'].str[:4]).transform('size')
cleanData.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,GroupId,GroupSize
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,0001,1
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,0002,1
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,0003,2
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,0003,2
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,0004,1


In [12]:
#Finding the largest group size in dataset (simply)

cleanData['GroupSize'].max()

8

Okay, we are done with the first part where we explored passengerId (clean from the start), and Name (useless, so we delete column at the end)

We also added groupId and groupSize columns since that gives us important data

### 2) Homeplanet and Destination

These are very important and interesting datapoints to explore (passengers from Europa is crazy, Europa is a moon of Jupiter theorized to have water under its icy surface and is a potential place where life could exist)

In [13]:
#Let's know the possible planets and destinations

print(f"Possible HomePlanets are {data['HomePlanet'].unique()}")
print(f"Possible destinations are {data['Destination'].unique()}")

Possible HomePlanets are ['Europa' 'Earth' 'Mars' nan]
Possible destinations are ['TRAPPIST-1e' 'PSO J318.5-22' '55 Cancri e' nan]


In [14]:
#Number of Nan values

print(f"Number of NaN values in HomePlanet is {data['HomePlanet'].isna().sum()}")
print(f"Number of NaN values in Destination is {data['Destination'].isna().sum()}")

Number of NaN values in HomePlanet is 201
Number of NaN values in Destination is 182


We can already see from above data viewing that people from same group need not necessarily have the same destination, but they might be from the same home planet, let's test that hypothesis:

In [15]:
# Are people in same group from the same home planet?

cleanData.groupby(cleanData['GroupId']).filter(lambda x: (x['HomePlanet'].nunique(dropna = True) != 1) & (x['HomePlanet'].nunique(dropna = True) != 0))

#Yes, people from the same group are from the same home planet, we can use this to fill in some NaN values.

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,GroupId,GroupSize


In [16]:
#Let's check if there's any case where everyone in group has NaN HomePlanet value

cleanData.groupby(cleanData['GroupId']).filter(lambda x: ((x['HomePlanet'].isna() == True).all()) & ((x['GroupSize'] > 1).all()))

#Only these 2 bozos, let's deal with them separately later

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,GroupId,GroupSize
6917,7336_01,NaN,False,F/1411/S,TRAPPIST-1e,30.0,False,2.0,0.0,5.0,1.0,962.0,Dianie Lambles,False,7336,2
6918,7336_02,NaN,False,F/1411/S,TRAPPIST-1e,26.0,False,0.0,0.0,902.0,0.0,0.0,Tanley Leodger,False,7336,2


In [17]:
#Let's find all NaN values of people in group, to fill them in with the same HomePlanet as the rest of the group

cleanData[((cleanData['GroupSize'] > 1) & (cleanData['HomePlanet'].isna())) & (cleanData['GroupId'] != '7336')]
#We can deal with 90 such values

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,GroupId,GroupSize
59,0064_02,NaN,True,E/3/S,TRAPPIST-1e,33.0,False,0.0,0.0,NaN,0.0,0.0,Colatz Keen,True,0064,2
113,0119_01,NaN,False,A/0/P,TRAPPIST-1e,39.0,False,0.0,2344.0,0.0,65.0,6898.0,Batan Coning,False,0119,2
405,0444_02,NaN,False,F/99/P,TRAPPIST-1e,26.0,True,1869.0,0.0,136.0,0.0,0.0,Colal Curte,False,0444,2
407,0445_02,NaN,False,E/25/S,TRAPPIST-1e,26.0,False,0.0,0.0,0.0,77.0,715.0,Rice Marshopper,False,0445,2
438,0470_02,NaN,True,F/86/S,TRAPPIST-1e,36.0,False,0.0,0.0,0.0,0.0,0.0,Jurs Binie,True,0470,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8353,8938_01,NaN,True,F/1729/S,TRAPPIST-1e,22.0,False,0.0,0.0,0.0,0.0,0.0,Quites Must,True,8938,2
8383,8961_04,NaN,False,F/1839/P,TRAPPIST-1e,21.0,False,0.0,6.0,0.0,593.0,0.0,Dandy Blancoy,False,8961,5
8454,9030_02,NaN,False,G/1465/P,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,298.0,511.0,Stace Ericksonga,False,9030,2
8489,9072_01,NaN,True,F/1758/S,TRAPPIST-1e,23.0,False,0.0,0.0,0.0,0.0,0.0,Gork Creke,True,9072,3


In [18]:
#Dealing with NaN HomePlanet values of group members

#To replace HomePlanet with first non-NaN value for all elements in same group
cleanData['HomePlanet'] = cleanData.groupby(cleanData['GroupId'])['HomePlanet'].transform('first')

#Check again
cleanData[((cleanData['GroupSize'] > 1) & (cleanData['HomePlanet'].isna())) & (cleanData['GroupId'] != '7336')]
#Successfully cleaned Yayyyy

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,GroupId,GroupSize


In [19]:
#Check how many NaN values left
print(f"Number of NaN values in HomePlanet is {cleanData['HomePlanet'].isna().sum()}")

Number of NaN values in HomePlanet is 111


In [20]:
#Ok to deal with those bozos, let's first see distribution of homeplanets

cleanData['HomePlanet'].value_counts(normalize = True)

#It is 50% likely they are from Earth

HomePlanet
Earth     0.539967
Europa    0.251806
Mars      0.208227
Name: proportion, dtype: float64

In [21]:
#I think it is justifiable to fill all the na values in HomePlanet with 'Earth' (including our bozos)
#We have already done the smart filling that we could from our observed pattern

#Filling Na values with mode (Earth)
cleanData['HomePlanet'] = cleanData['HomePlanet'].fillna('Earth')

#Verify
print(f"Number of NaN values in HomePlanet is {cleanData['HomePlanet'].isna().sum()}")

Number of NaN values in HomePlanet is 0


In [22]:
#Now let's deal with NaN values in Destination

cleanData['Destination'].value_counts(normalize = True)

#Over 69% of people headed to TRAPPIST-1e

Destination
TRAPPIST-1e      0.694983
55 Cancri e      0.211491
PSO J318.5-22    0.093526
Name: proportion, dtype: float64

In [23]:
#Think its very fair to fill Destination NaN values with TRAPPIST-1e

#Filling values
cleanData['Destination'] = cleanData['Destination'].fillna('TRAPPIST-1e')

#Verification
print(f"Number of NaN values in Destination is {cleanData['Destination'].isna().sum()}")

Number of NaN values in Destination is 0


In [24]:
#Final Verification that we didnt mess smtg up

print(f"Possible HomePlanets are {cleanData['HomePlanet'].unique()}")
print(f"Possible destinations are {cleanData['Destination'].unique()}")

Possible HomePlanets are ['Europa' 'Earth' 'Mars']
Possible destinations are ['TRAPPIST-1e' 'PSO J318.5-22' '55 Cancri e']


Okay, now we are done with the 2nd part where we finished cleaning up the HomePlanet and Destination columns.

First we smartly filled in some of the HomePlanet NaN values after noticing that people in same group always came from same HomePlanet (and noticed that same logic doesnt apply for Destination), then we filled the rest of the NaN values with the mode of HomePlanet and Destination values.

### 3) Cabin

Cabin values have some cryptic pattern that we will try to understand, which might help us further understand the situation on the passenger ship

In [25]:
#Let's try to figure out the pattern behind the cabin values

#There are clearly 3 different parts to it, and it seems like the last part can take values from {P, S}, and first part can take values from A to G (maybe)
#To test this let's split the string into 3 parts in the clean df and test our hypothesis

#Temp df
#tempData = cleanData.copy()
#Was originally done in temporary df but it's important so just added these new columns in our cleanData set

#Splitting string into 3 columns based on / 

cleanData[['Cabin P1', 'Cabin P2', 'Cabin P3']] = cleanData['Cabin'].str.split("/", expand = True)

cleanData.head(10)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,GroupId,GroupSize,Cabin P1,Cabin P2,Cabin P3
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,0001,1,B,0,P
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,0002,1,F,0,S
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,0003,2,A,0,S
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,0003,2,A,0,S
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,0004,1,F,1,S
5,0005_01,Earth,False,F/0/P,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.0,Sandie Hinetthews,True,0005,1,F,0,P
6,0006_01,Earth,False,F/2/S,TRAPPIST-1e,26.0,False,42.0,1539.0,3.0,0.0,0.0,Billex Jacostaffey,True,0006,2,F,2,S
7,0006_02,Earth,True,G/0/S,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,NaN,Candra Jacostaffey,True,0006,2,G,0,S
8,0007_01,Earth,False,F/3/S,TRAPPIST-1e,35.0,False,0.0,785.0,17.0,216.0,0.0,Andona Beston,True,0007,1,F,3,S
9,0008_01,Europa,True,B/1/P,55 Cancri e,14.0,False,0.0,0.0,0.0,0.0,0.0,Erraiam Flatic,True,0008,3,B,1,P


In [26]:
#Let's analyze these 3 parts, starting from the first

cleanData['Cabin P1'].unique()
#Okay we were sorta wrong, part 1 takes values from {A, B, C, D, E, F, G, T} (what we hypothesized + T (interesting))

array(['B', 'F', 'A', 'G', nan, 'E', 'D', 'C', 'T'], dtype=object)

In [27]:
#Let's see how often this T comes up (maybe its an error?)

cleanData['Cabin P1'].value_counts()

#Truly could be an error, we will keep that into account for later

Cabin P1
F    2794
G    2559
E     876
B     779
C     747
D     478
A     256
T       5
Name: count, dtype: int64

In [28]:
#Let's see part 3 next, as that's interesting

cleanData['Cabin P3'].unique()

#Just like we predicted, what could P and S mean? hmm

array(['P', 'S', nan], dtype=object)

In [29]:
#Now let's see part 2

#Let's first convert Cabin P2 into integers

cleanData['Cabin P2'] = cleanData['Cabin P2'].astype('Int64')

cleanData['Cabin P2'].unique()
#Perhaps some sort of cabin number?

<IntegerArray>
[   0,    1,    2,    3, <NA>,    4,    5,    6,    8,    9,
 ...
 1782, 1884,  353, 1886, 1498, 1888, 1891, 1892, 1893, 1894]
Length: 1818, dtype: Int64

In [30]:
#Let's see if it is continous (all values betweeen 0 and 1894 are present?)

#initial = 0
#for i in range(1895):
#    if i not in cleanData['Cabin P2'].unique():
#        if initial == 0:
#         print(f"The first value missing between 0 and 1894 is {i}")
#         initial += 1
#        else:
#         initial += 1
#print(f"The number of values missing between 0 and 1894 is {initial}")

#I actually have no idea why i did this 
#i dont think it matters anyways since we know that some of the data is split into train and test (some passengerId are missing)
        

In [31]:
#Assuming Cabin P2 is some sort of cabin number, maybe ppl in each cabin shud have similar number of people?
# (or maybe A-G denotes size of cabin)

cleanData['Cabin P2'].value_counts()

#Each cabin no doesnt have similar no of ppl, but the range of number of people in a cabin ranges from 1 to 28

Cabin P2
82      28
86      22
19      22
176     21
56      21
        ..
1839     1
1848     1
1847     1
1749     1
1782     1
Name: count, Length: 1817, dtype: Int64

In [32]:
#Hypothesis: One room number cannot have 2 different Cabin P1 values (A-G) (since we think A-G symbolizes size of cabin)

cleanData.groupby(cleanData['Cabin P2']).filter(lambda x: x['Cabin P1'].nunique() != 1)

#Ok i was very much wrong :( (n thats ok :'))

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,GroupId,GroupSize,Cabin P1,Cabin P2,Cabin P3
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,0001,1,B,0,P
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,0002,1,F,0,S
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,0003,2,A,0,S
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,0003,2,A,0,S
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,0004,1,F,1,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False,9276,1,A,98,P
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False,9278,1,G,1499,S
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True,9279,1,G,1500,S
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False,9280,2,E,608,S


In [33]:
#Since A-G comes before cabin number, maybe its like 'block name' 
# and each block have their own set of numbers from 0 like that or smtg 
#Also each block gets progressively bigger (interestingly) (I WAS WRONG, F is bigger than G welp im blind)

#To test this: There should be all values from A-G (maybe even T if it actually means something) with cabin P2 = 0 (unless missing)

cleanData[cleanData['Cabin P2'] == 0]

#Yes, cabin no = 0 exists for all cabin block numbers from A-G AND even T

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,GroupId,GroupSize,Cabin P1,Cabin P2,Cabin P3
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,0001,1,B,0,P
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,0002,1,F,0,S
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,0003,2,A,0,S
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,0003,2,A,0,S
5,0005_01,Earth,False,F/0/P,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.0,Sandie Hinetthews,True,0005,1,F,0,P
7,0006_02,Earth,True,G/0/S,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,NaN,Candra Jacostaffey,True,0006,2,G,0,S
19,0017_01,Earth,False,G/0/P,TRAPPIST-1e,0.0,False,0.0,0.0,0.0,0.0,0.0,Lyde Brighttt,True,0017,2,G,0,P
21,0020_01,Earth,True,E/0/S,TRAPPIST-1e,1.0,False,0.0,0.0,0.0,0.0,0.0,Almary Brantuarez,False,0020,6,E,0,S
22,0020_02,Earth,True,E/0/S,55 Cancri e,49.0,False,0.0,0.0,0.0,0.0,0.0,Glendy Brantuarez,False,0020,6,E,0,S
23,0020_03,Earth,True,E/0/S,55 Cancri e,29.0,False,0.0,0.0,NaN,0.0,0.0,Mollen Mcfaddennon,False,0020,6,E,0,S


In [34]:
#Let's now investigate the mystery of the T cabin block (maybe they are VIP, maybe they are like special engineers or the pilots?)

cleanData[cleanData['Cabin P1'] == 'T']

#Absolutely nothing about them seems unique

#It is most likely/most natural to think that the T group is a mistake
# Firstly, it is out of order A-G and then skipping to T? 
# And if they were some 5 special people, I would've liked to seen evidence of that in their data, which is not present

#Imo it is most likely that these 5 people just have a wrong value of cabin block name

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,GroupId,GroupSize,Cabin P1,Cabin P2,Cabin P3
1004,1071_01,Earth,False,T/0/P,TRAPPIST-1e,35.0,False,415.0,1328.0,0.0,14.0,60.0,Alraida Dingauge,False,1071,1,T,0,P
2254,2414_01,Europa,False,T/1/P,TRAPPIST-1e,42.0,False,0.0,1829.0,2.0,3133.0,2447.0,Mergak Headfair,False,2414,1,T,1,P
2734,2935_01,Europa,False,T/2/P,TRAPPIST-1e,33.0,False,0.0,28.0,0.0,6841.0,543.0,NaN,False,2935,1,T,2,P
2763,2971_01,Europa,False,T/3/P,TRAPPIST-1e,38.0,False,0.0,3135.0,0.0,26.0,3.0,Pent Sessiouse,True,2971,1,T,3,P
4565,4863_01,Europa,NaN,T/2/S,TRAPPIST-1e,37.0,False,1721.0,667.0,NaN,28.0,1362.0,Phacton Unsible,False,4863,1,T,2,S


In [35]:
#We know the mode of the cabin P1 is F but let's see the %es

cleanData['Cabin P1'].value_counts(normalize = True)

#32% F vs 30% G is quite close and thus quite difficult to consider F as a mode since the margin is so less
#Also it is clear that T is a MAJOR outlier, let's put them in F

Cabin P1
F    0.328938
G    0.301271
E    0.103132
B    0.091712
C    0.087944
D    0.056275
A    0.030139
T    0.000589
Name: proportion, dtype: float64

In [51]:
#Fixing the error of T:

cleanData['Cabin P1'] = cleanData['Cabin P1'].replace('T', 'F')
cleanData[cleanData['Cabin P1'] == 'T']

#We have fixed the error of T block number

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,GroupId,GroupSize,Cabin P1,Cabin P2,Cabin P3


In [37]:
#Check if all people in group are always in same cabin number (P1, P2 and P3)

cleanData.groupby(cleanData['GroupId']).filter(lambda x: (x['Cabin'].nunique(dropna = True) != 1) & (x['Cabin'].nunique(dropna = True) != 0))

#The result is shockingly, NO (that too for a LARGE number of them (1210))

#This is quite troublesome, maybe it implies that either
# 1) The group members arent even like friends like they might be grouped together based on something else completely idk
# 2) The people just chose to not be with their group members for some reason
# 3) The people didnt have a choice on which cabin they'd get (unlikely)

#I guess option (2) is most likely, but this means that groupId doesn't really influence cabin number at all
#Atleast in these examples, same groupId were always in same P/S tho, let's check if thats universally true next

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,GroupId,GroupSize,Cabin P1,Cabin P2,Cabin P3
6,0006_01,Earth,False,F/2/S,TRAPPIST-1e,26.0,False,42.0,1539.0,3.0,0.0,0.0,Billex Jacostaffey,True,0006,2,F,2,S
7,0006_02,Earth,True,G/0/S,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,NaN,Candra Jacostaffey,True,0006,2,G,0,S
19,0017_01,Earth,False,G/0/P,TRAPPIST-1e,0.0,False,0.0,0.0,0.0,0.0,0.0,Lyde Brighttt,True,0017,2,G,0,P
20,0017_02,Earth,False,F/6/P,55 Cancri e,14.0,False,412.0,0.0,1.0,0.0,679.0,Philda Brighttt,False,0017,2,F,6,P
58,0064_01,Mars,True,F/14/S,TRAPPIST-1e,15.0,False,0.0,0.0,0.0,0.0,0.0,NaN,True,0064,2,F,14,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8653,9231_01,Mars,True,F/1888/P,55 Cancri e,2.0,False,0.0,0.0,0.0,0.0,0.0,Denuts Paie,True,9231,3,F,1888,P
8654,9231_02,Mars,False,F/1888/P,TRAPPIST-1e,0.0,False,0.0,0.0,0.0,0.0,0.0,Walls Paie,True,9231,3,F,1888,P
8655,9231_03,Mars,False,E/592/P,TRAPPIST-1e,22.0,False,0.0,0.0,0.0,0.0,0.0,Cus Paie,False,9231,3,E,592,P
8682,9272_01,Earth,False,G/1507/P,TRAPPIST-1e,26.0,False,240.0,242.0,510.0,0.0,0.0,Ireene Simson,True,9272,2,G,1507,P


In [38]:
#Check if same group number = same cabin P3 value

cleanData.groupby(cleanData['GroupId']).filter(lambda x: (x['Cabin P3'].nunique(dropna = True) != 1) & (x['Cabin P3'].nunique(dropna = True) != 0))

#Yes! Everyone in group always has same Cabin P3 value (P/S) but not necessarily the same Cabin P1 or P2 values 
# Very interseting, what could Cabin P3 refer to? (no clue)

#But we will definitely use this fact to fill P3 values yay

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,GroupId,GroupSize,Cabin P1,Cabin P2,Cabin P3


Okay now let's commence filling of NaN values. Just to note what all we learnt about the pattern in Cabin numbers

Firstly, the first cabin letter likely corresponds to a 'block' or a 'location' in the ship where rooms are (from A-G)

Secondly, the number likely corresponds to cabin number in the respective block

Thirdly, the last value P/S (still no clue waht it means), BUT people in same group have same P/S Value!! (helpful)

In [39]:
#Wherever Cabin is NaN, cabin P1, P2 and P3 will also take up NaN values

print(f"Number of NaN values in cabin data is {cleanData['Cabin'].isna().sum()}")

Number of NaN values in cabin data is 199


In [40]:
#I think it is fair to fill up values of P1 with mode (F) for the 199 NaN values

cleanData['Cabin P1'] = cleanData['Cabin P1'].fillna('F')

print(f"Number of NaN values in cabin P1 data is {cleanData['Cabin P1'].isna().sum()}")

Number of NaN values in cabin P1 data is 0


In [ ]:
#We need to somehow fill NaN values in Cabin P2 with a random value from the range of values in their respective block 
#This seems like the most logical way to do it
#It is not enough to fill from range of values of all cabin numbers since we know that different blocks have different ranges of cabin numbers

#Set a randomstate to make sure results are reproducible
np.random.seed(69)

#Ok we need a custom function that does this task for us since one doesnt exist:
#We need for a column that we get its NaN positions
#We need to generate (number of NaN) number of random elements from the rest of the data
#We need to fill in the calculated NaN positions with the generated random non NaN values

def randomFill(column):
    #Was giving me some warning later so SGMRW = stop giving me random warnings
    SGMRW = column.copy()
    
    #Get indices of NaN values
    indices = SGMRW[SGMRW.isna()].index
    
    #Get random fillable values
    randomVals = np.random.choice(SGMRW.dropna(), size = SGMRW.isna().sum(), replace = True)

    #Fill in the random values
    SGMRW.loc[indices] = randomVals
    return SGMRW

#Perform this process groupwise
cleanData['Cabin P2'] = cleanData.groupby(cleanData['Cabin P1'])['Cabin P2'].transform(lambda x: randomFill(x))

print(f"Number of NaN values in cabin P2 data is {cleanData['Cabin P2'].isna().sum()}")

Number of NaN values in cabin P2 data is 0


In [42]:
#First let's fill the NaN P3 values of people in same group smartly similar to how we did before

print(f"Number of NaN values in cabin P3 data is {cleanData['Cabin P3'].isna().sum()}")

#Same smart filling of filling everyone in group with same Cabin P3 value (since they all have same cabin P3 value)
cleanData['Cabin P3'] = cleanData.groupby(cleanData['GroupId'])['Cabin P3'].transform('first')

print(f"Number of NaN values in cabin P3 data is {cleanData['Cabin P3'].isna().sum()}")
#99 more to go which can't be resolved in this way (including those 2 bozos from earlier lmao)

Number of NaN values in cabin P3 data is 199
Number of NaN values in cabin P3 data is 99


In [43]:
#Checking distribution of Cabin P3 values to see if there is a mode or if its more 50-50 (im guessing itd be 50-50ish)

cleanData['Cabin P3'].value_counts(normalize = True)

#haha i knew it, so we will randomly fill these in as well 

Cabin P3
S    0.505353
P    0.494647
Name: proportion, dtype: float64

In [44]:
#Filling in the rest of the Cabin P3 NaN values using our randomFill function

#Using our op function we made up just before to fill in (since is already around 50-50 this basically same as picking from P and S 50-50)
cleanData['Cabin P3'] = randomFill(cleanData['Cabin P3'])

print(f"Number of NaN values in cabin P3 data is {cleanData['Cabin P3'].isna().sum()}")

Number of NaN values in cabin P3 data is 0


In [50]:
#Now that we have cleaned all the Cabin P1, P2, P3 values, let's put it all together by fixing the cabin values

cleanData['Cabin'] = cleanData['Cabin P1'].astype(str) + '/' + cleanData['Cabin P2'].astype(str) + '/' + cleanData['Cabin P3'].astype(str)
print(f"Number of NaN values in cabin data is {cleanData['Cabin'].isna().sum()}")
cleanData.head()

Number of NaN values in cabin data is 0


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,GroupId,GroupSize,Cabin P1,Cabin P2,Cabin P3
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,0001,1,B,0,P
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,0002,1,F,0,S
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,0003,2,A,0,S
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,0003,2,A,0,S
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,0004,1,F,1,S


So here we first figured out that: (After splitting cabin into 3 diff columns)
- the first letter part of cabin corresponds to some sort of 'block'
- There was possibly a corruption that led to a outlier 'T' block (so we replaced it with the mode F)
- the second number in the cabin is some sort of cabin number (independent for each block)
- there are 2 types of cabin rooms P or S (not sure what it means)
- each group travelling might not be in the same cabin room BUT always has same type of cabin room (P or S)

Then for filling:
- We filled first letter NaN values with the mode (F)
- We first second number (cabin number) NaN values with a random filling from the possible cabin number values in their respective blocks
- For the third cabin type, we first smart filled based on the fact that people in same group had same cabin type, and then we randomly filled (sinced the distribution was 50-50ish) using same custom function that we developed earlier

### 4) Cryosleep and VIP